In [1]:
import os
if os.getcwd() == '/home/user/code':
    os.chdir('/home/user/code/nlp2024_ClefTask4SOTA')

In [2]:
from TexSoup import TexSoup
import re

def find_sections(tex):
    return [match.group(1) for match in re.finditer("\\\section\{(.*)\}", tex)]


def extract_content(node, name):
    if node:
        return ((name, node.contents[0]))


def section_split(tex):
    """returns: List of tuples (section_name, section_text)"""
    doc_text = tex
    sections = []
    # parsing with TexSoup
    # TODO: get rid of this and use regex only (brittle)
    try:
        soup = TexSoup(tex, tolerance=1)
        if title:=extract_content(soup.title, "title"):
            sections.append(title)
        if abstract:=extract_content(soup.abstract, "abstract"):
            sections.append(abstract)
        sections.append(("tables", "\n".join([str(node) for node in soup.find_all("table")]))) if soup.find_all("table") else None
    except:
        pass # could not parse tex

    # extract latex sections and corresponding text
    prev_section = "pre"

    for section in find_sections(tex):
        section_text, doc_text = doc_text.split(f"\\section{{{section}}}", 1)
        sections.append((prev_section, section_text))
        prev_section = section
    sections.append((prev_section, doc_text))
    return sections
    # except:
    #     return (("full", tex))
    


In [6]:
import ollama


def pass_to_ollama(prompt, model):
    try:
        res = ollama.generate(model=model, prompt=prompt, options={"temperature": 0})
        return res["response"]
    except Exception as ex:
        print(ex)
        return f"ollama error: {ex}"

def extract_tdms(tex):
    return f'If the text reports benchmark leaderboard results, extract the reported Tasks, Datasets, Metrics and corresponding Scores.\
     \n \
    Text: \n {tex} \n \
    Return the tasks, datasets, metrics and scores as reported in the text in a JSON array, \
    for example [{{"Task": "example Task 1", "Dataset": "example Dataset 1", "Metric": example metric 1", "Score": "score"}}, {{"Task": "example Task 1","Dataset": "example Dataset 2", "Metric": example metric 2", "Score": "score"}}] and provide the JSON Array only. \n \
    Do not include precision information in the reported score. \
    Entries: '

def convert_tdms_to_tuple(model_output_parsed):
    tuples = []
    for item in model_output_parsed:
        try:
            t = ((item["Task"], item["Dataset"],item["Metric"],item["Score"]))
            tuples.append(t)
        except:
            # parse error, ignore instance
            pass
    return tuples

def format_tdms(tuples):
    """make unique, format as string"""
    unique = set(tuples)
    dicts = [{"LEADERBOARD": {
        "Task": t,
        "Dataset":d,
        "Metric":m,
        "Score":s
    }} for t,d,m,s in unique]
    return str(dicts)


In [ ]:
# import json
# import tqdm
# from tqdm import tqdm
# from src.dataset import TDMSDataset, PATH, UNANSWERABLE, LogResult
# from datetime import datetime

# # Run on validation data
# valid_dataset = TDMSDataset(PATH.VAL)
# model = "llama3:70b"


# now_str = datetime.now().strftime('%m%d%Y-%H%M%S')
# model_str = model.replace(':', '_')
# run_id = f"baseline-tdms-valid_{model_str}-{now_str}"

# llama3_fn = lambda prompt: pass_to_ollama(prompt, model)


# logger = LogResult(run_id, do_write=True)
# indexes = len(valid_dataset)

# for i in tqdm(range(indexes)):
#     f, tex, ground_truth = valid_dataset.__getitem__(i)
#     found_tdms = []
#     sections = section_split(str(tex))
#     for section_name, section_text in sections:
#         response = llama3_fn(extract_tdms(section_text))

#         try:
#             response = json.loads(response)
#             parsed = convert_tdms_to_tuple(response)
#             found_tdms= [*found_tdms, *parsed]
#         except:
#             pass # no tuples found in section

#     # print(found_tdms)
#     if not found_tdms:
#         annotation = UNANSWERABLE # found_tdms are empty -> unanswerable
#     else:
#         # dedupe and format
#         annotation = format_tdms(found_tdms)
#     # log
#     logger.log(f, annotation)

# df = logger.save()


# df

  0%|                                                                                                  | 0/100 [00:00<?, ?it/s]

# Run on Test

In [7]:
import json
import tqdm
from tqdm import tqdm
from src.dataset import TDMSDataset, PATH, UNANSWERABLE, LogResult
from datetime import datetime

# Run on validation data
dataset = TDMSDataset(PATH.TEST)
model = "llama3:70b"


now_str = datetime.now().strftime('%m%d%Y-%H%M%S')
model_str = model.replace(':', '_')
run_id = f"baseline-tdms-test_{model_str}-{now_str}"

llama3_fn = lambda prompt: pass_to_ollama(prompt, model)


logger = LogResult(run_id, do_write=True)
indexes = len(dataset)

for i in tqdm(range(indexes)):
    f, tex, ground_truth = dataset.__getitem__(i)
    found_tdms = []
    sections = section_split(str(tex))
    for section_name, section_text in sections:
        response = llama3_fn(extract_tdms(section_text))

        try:
            response = json.loads(response)
            parsed = convert_tdms_to_tuple(response)
            found_tdms= [*found_tdms, *parsed]
        except:
            pass # no tuples found in section

    # print(found_tdms)
    if not found_tdms:
        annotation = UNANSWERABLE # found_tdms are empty -> unanswerable
    else:
        # dedupe and format
        annotation = format_tdms(found_tdms)
    # log
    logger.log(f, annotation)

df = logger.save()


df

  0%|                                                                                                  | 0/789 [00:00<?, ?it/s]


NameError: name 'section_split' is not defined